In [1]:
import tensorflow as tf

class EntropySGDWrapper(tf.keras.Model):
    def __init__(
        self,
        base_model,
        loss_fn,
        metrics=None,
        learning_rate=1e-3,
        momentum=0.0,
        nesterov=False,
        sgld_step=0.1,
        L=20,
        gamma=0.03,
        sgld_noise=1e-4,
        alpha=0.75,
        scoping=1e-3,
        name="entropy_sgd_wrapper",
    ):
        super().__init__(name=name)
        self.base = base_model
        self.loss_fn = loss_fn
        self._metric_list = metrics or []

        # Hyperparameters (non-trainable variables for easy assignment if needed)
        self.outer_lr   = tf.Variable(learning_rate, trainable=False, dtype=tf.float32, name="outer_lr")
        self.momentum   = tf.Variable(momentum,     trainable=False, dtype=tf.float32, name="momentum")
        self.nesterov   = bool(nesterov)

        self.sgld_step  = tf.Variable(sgld_step,    trainable=False, dtype=tf.float32, name="sgld_step")
        self.L          = int(L)
        self.gamma      = tf.Variable(gamma,        trainable=False, dtype=tf.float32, name="gamma")
        self.sgld_noise = tf.Variable(sgld_noise,   trainable=False, dtype=tf.float32, name="sgld_noise")
        self.alpha      = tf.Variable(alpha,        trainable=False, dtype=tf.float32, name="alpha")
        self.scoping    = tf.Variable(scoping,      trainable=False, dtype=tf.float32, name="scoping")

        # Shadow/state variables (built lazily)
        self._built_shadows = False
        self._x_vars  = []   # slow weights (x)
        self._mu_vars = []   # EMA (mu)
        self._m_vars  = []   # momentum buffers
        self._first_batch = tf.Variable(True, trainable=False)


    @property
    def metrics(self):
        # Keras will reset these each epoch
        return self._metric_list

    def call(self, inputs, training=False):
        return self.base(inputs, training=training)

    def _build_shadows_if_needed(self):
        if self._built_shadows:
            return
        for v in self.base.trainable_variables:
            x  = tf.Variable(v, trainable=False, name=v.name.replace(":", "_") + "_x")
            mu = tf.Variable(v, trainable=False, name=v.name.replace(":", "_") + "_mu")
            m  = tf.Variable(tf.zeros_like(v), trainable=False, name=v.name.replace(":", "_") + "_m")
            self._x_vars.append(x)
            self._mu_vars.append(mu)
            self._m_vars.append(m)
        self._built_shadows = True

    @tf.function
    def train_step(self, data):
        # Make sure shadow vars exist (created in eager or build()).
        self._build_shadows_if_needed()

        x_batch, y_batch, sample_weight = tf.keras.utils.unpack_x_y_sample_weight(data)

        # One-time init of shadows from base weights (first batch only)
        if self._first_batch:
            for v, x, mu, m in zip(self.base.trainable_variables, self._x_vars, self._mu_vars, self._m_vars):
                x.assign(v)
                mu.assign(v)
                m.assign(tf.zeros_like(v))
            self._first_batch.assign(False)

        # Start inner loop from slow weights x
        for v, x in zip(self.base.trainable_variables, self._x_vars):
            v.assign(x)

        # Tensor rebinding (not Variable.assign) for gamma schedule
        gamma_local = tf.identity(self.gamma)
        growth = 1.0 + self.scoping

        # Inner SGLD loop
        for _ in tf.range(self.L):
            with tf.GradientTape() as tape:
                preds = self.base(x_batch, training=True)
                loss = (self.loss_fn(y_batch, preds, sample_weight=sample_weight)
                        if sample_weight is not None else self.loss_fn(y_batch, preds))
            grads = tape.gradient(loss, self.base.trainable_variables)

            for v, g, x, mu in zip(self.base.trainable_variables, grads, self._x_vars, self._mu_vars):
                if g is None:
                    continue
                dx_prime = g - gamma_local * (x - v)
                v.assign_sub(self.sgld_step * tf.cast(dx_prime, v.dtype))

                # Always add noise (scales to zero if sgld_noise == 0)
                noise = tf.random.normal(tf.shape(v), dtype=v.dtype)
                v.assign_add(tf.sqrt(self.sgld_step) * self.sgld_noise * noise)

                # EMA update
                mu.assign((1.0 - self.alpha) * mu + self.alpha * v)

            gamma_local = gamma_local * growth  # rebind tensor

        # Outer update on slow weights x (SGD/Nesterov)
        for x, mu, m in zip(self._x_vars, self._mu_vars, self._m_vars):
            grad_outer = x - mu
            v_buf = self.momentum * m - self.outer_lr * grad_outer
            m.assign(v_buf)
            if self.nesterov:
                x.assign(x + self.momentum * v_buf - self.outer_lr * grad_outer)
            else:
                x.assign(x + v_buf)

        # Sync fast weights to updated slow weights
        for v, x in zip(self.base.trainable_variables, self._x_vars):
            v.assign(x)

        # Final loss/metrics for logging
        preds = self.base(x_batch, training=True)
        final_loss = (self.loss_fn(y_batch, preds, sample_weight=sample_weight)
                      if sample_weight is not None else self.loss_fn(y_batch, preds))
        logs = {"loss": final_loss}
        for m in self.metrics:
            m.update_state(y_batch, preds, sample_weight=sample_weight)
            logs[m.name] = m.result()
        return logs



    @tf.function
    def test_step(self, data):
        x_batch, y_batch, sample_weight = tf.keras.utils.unpack_x_y_sample_weight(data)
        preds = self.base(x_batch, training=False)
        loss  = self.loss_fn(y_batch, preds, sample_weight=sample_weight) \
                if sample_weight is not None else self.loss_fn(y_batch, preds)

        logs = {"loss": loss}
        for m in self.metrics:
            m.update_state(y_batch, preds, sample_weight=sample_weight)
            logs[m.name] = m.result()
        return logs


E0000 00:00:1755554296.443108    7095 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755554296.464758    7095 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755554296.620811    7095 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755554296.620841    7095 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755554296.620846    7095 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755554296.620849    7095 computation_placer.cc:177] computation placer already registered. Please check linka

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

ImportError: Keras cannot be imported. Check that it is installed.

In [4]:
# 1) Load & preprocess MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Flatten to 784 and normalize to [0,1]
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test  = x_test.reshape(-1, 784).astype("float32") / 255.0

# Optional: create a validation split from train
val_split = 0.1
num_val = int(len(x_train) * val_split)
x_val, y_val = x_train[:num_val], y_train[:num_val]
x_train2, y_train2 = x_train[num_val:], y_train[num_val:]

# tf.data pipelines
batch_size = 128
train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train2, y_train2))
    .shuffle(10000)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((x_val, y_val))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
test_ds = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# 2) Define the base model (simple MLP)
base = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(784,)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# 3) Wrap with EntropySGD
entropy_model = EntropySGDWrapper(
    base_model=base,
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")],
    learning_rate=1e-3,
    momentum=0.9,
    nesterov=True,
    sgld_step=0.1,   # inner-loop step size
    L=20,            # inner SGLD steps per batch
    gamma=0.03,      # coupling strength
    sgld_noise=1e-4, # thermal noise
    alpha=0.75,      # EMA of x'
    scoping=1e-3,    # gamma scoping factor
)




In [5]:
# Force-build base model variables
_ = base(tf.zeros((1, 784), dtype=tf.float32))

# Create shadow variables eagerly
entropy_model._build_shadows_if_needed()

# Initialize shadows from base weights once (eager)
for v, x, mu, m in zip(entropy_model.base.trainable_variables,
                       entropy_model._x_vars,
                       entropy_model._mu_vars,
                       entropy_model._m_vars):
    x.assign(v)
    mu.assign(v)
    m.assign(tf.zeros_like(v))
entropy_model._first_batch.assign(False)



<tf.Variable 'UnreadVariable' shape=() dtype=bool, numpy=False>

In [6]:
entropy_model.compile()


In [7]:
history = entropy_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1,
)


Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.5540 - loss: 1.0592 - val_acc: 0.8748 - val_loss: 0.4908
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - acc: 0.8707 - loss: 0.4609 - val_acc: 0.8978 - val_loss: 0.3485
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - acc: 0.8892 - loss: 0.3785 - val_acc: 0.9070 - val_loss: 0.3045
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - acc: 0.8979 - loss: 0.3422 - val_acc: 0.9122 - val_loss: 0.2820
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - acc: 0.9023 - loss: 0.3193 - val_acc: 0.9162 - val_loss: 0.2666
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - acc: 0.9092 - loss: 0.3038 - val_acc: 0.9208 - val_loss: 0.2554
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - acc: 0.9120 - loss: 0.2901 - val_acc: 0.9235 - val_loss: 0.2455
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - acc: 0.9162 - loss: 0.2798 - val_acc: 0.9258 - val_loss: 0.2396
Epoch 9/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - a

In [8]:
# 6) Evaluate
test_logs = entropy_model.evaluate(test_ds, return_dict=True, verbose=1)
print("Test metrics:", test_logs)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9150 - loss: 0.2458
Test metrics: {'loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.10337677597999573>, 'acc': <tf.Tensor: shape=(), dtype=float32, numpy=0.9279000163078308>}
